In [ ]:
%matplotlib inline

# Python 2.x / 3.x compatibility
from __future__ import division, print_function

#Import modules
import pandas as pd
import numpy as np
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re

#import csv
import glob as gb

#import pathlib

import datetime
import sqlite3

from df2gspread import df2gspread as d2g

mpl.rcParams['figure.figsize'] = (16, 9)
pd.options.display.max_rows = 200

In [ ]:
import support
# from imp import reload
# reload(support)

# Parse compatibility Matrix

In [ ]:
compat_matrix = support.parse_compatibility_matrix()

In [ ]:
compat_matrix.head()

In [ ]:
compat_matrix['Has_Docker'].value_counts()

In [ ]:
compat_matrix[compat_matrix['Has_Docker']].tail()

In [ ]:
# Count Number of OpenStudio versions within each E+ version
compat_matrix.groupby('E+')['OpenStudio'].count()

In [ ]:
# Export to CSV
# compat_matrix.to_csv('compat_matrix.csv')

# Fix permissions and skin down the fuelcell OSW

In [ ]:
# Skinning it down is done in the model_tests.rb now
help(support.cleanup_bloated_osws)

The permissions stuff is done in the launch docker shell scripts

If you want to do it manually

Need to do:
    
    sudo chown -R $USER * 
    sudo find . -type f -exec chmod 664 {} \;

# Parse out.osw

In [ ]:
df_files = support.find_info_osws()

In [ ]:
df_files.head()

## Output the test status: Fail/Success/Blank

In [ ]:
from imp import reload
reload(support)

In [ ]:
# Prepare the dataframe
success = support.success_sheet(df_files)

In [ ]:
def background_colors(val):
    s = 'background-color: {}'
    if val == 'Fail':
        return s.format('#F4C7C3')
    elif val == '':
        return s.format('#f2e2c1')
    return ''

def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

styles = [
    hover(),
    dict(selector="td", props=[#("font-size", "150%"),
                               ("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "bottom")])
]

(success.style.applymap(background_colors).set_table_styles(styles)
          .set_caption("Test Success"))

In [ ]:
success_filt = success.loc[success[(success == '').sum(axis=1) >= 1].index.get_level_values(0).tolist()]
filt = success_filt[success_filt[('8.8.0', '2.4.1')] == 'Fail'].index.get_level_values(0).tolist()

(success.loc[filt].style.applymap(background_colors).set_table_styles(styles)
        .set_caption("Test Success"))

### Export to Google

In [ ]:
spreadsheet = '/EffiBEM&NREL-Regression-Test_Status'
wks_name = 'Test_Status'
d2g.upload(success.T.reset_index().T.reset_index(),
           gfile=spreadsheet, wks_name=wks_name,
           row_names=False, col_names=False)

## Output Missing tests: ruby versus osm

In [ ]:
test_impl = support.test_implemented_sheet(df_files=df_files, success=success,
                                   only_for_mising_osm=False)

In [ ]:
test_impl[~test_impl['osm']]

In [ ]:
spreadsheet = '/EffiBEM&NREL-Regression-Test_Status'
    wks_name = 'Tests_Implemented'
    d2g.upload(test_impl,
               gfile=spreadsheet, wks_name=wks_name,
               row_names=True, col_names=True)

## Ouput the total_site_energy (kBTU)

In [ ]:
site_kbtu = df_files.applymap(support.parse_total_site_energy)

In [ ]:
spreadsheet = '/EffiBEM&NREL-Regression-Test_Status'
wks_name = 'SiteKBTU'
d2g.upload(site_kbtu.T.reset_index().T.reset_index().fillna(''),
           gfile=spreadsheet, wks_name=wks_name,
           # Skip first row
           start_cell='A1',
           row_names=False, col_names=False)

## Output the rolling percent difference of total kBTU from one version to the next

In [ ]:
#site_kbtu.pct_change(axis=1).reset_index()

In [ ]:
spreadsheet = '/EffiBEM&NREL-Regression-Test_Status'
wks_name = 'SiteKBTU_Percent_Change'
d2g.upload(site_kbtu.pct_change(axis=1).T.reset_index().T.reset_index().fillna(''),
           gfile=spreadsheet, wks_name=wks_name,
           row_names=False, col_names=False)

## Difference in end use

In [ ]:
over_5pct = (site_kbtu.pct_change(axis=1).abs() > 0.005).sum(axis=0).to_frame()
over_5pct.columns = ['Count (ABS(pct_diff) > 0.5%)']

In [ ]:
site_kbtu.pct_change(axis=1).abs().describe()

In [ ]:
over_5pct

In [ ]:
over_5pct = (site_kbtu.pct_change(axis=1).abs() > 0.005).sum(axis=1).to_frame()
over_5pct.columns = ['Count (ABS(pct_diff) > 0.005)']

In [ ]:
over_5pct.replace(0, np.nan).dropna().sort_values('Count (ABS(pct_diff) > 0.005)', ascending=False)

In [ ]:
version_1 = '2.1.0'
version_2 = '2.2.1'


all_diffs = {}
failed = {}
for index, row in  df_files.T.reset_index(level=0, drop=True).T.iterrows():
    diff_ok = True
    try:
        cleaned_end_use_2 = support.parse_end_use(row[version_2])
        ok2 = True
    except:
        cleaned_end_use_2 = 'Failed'
        diff_ok = False
        ok2 = False
    try:
        cleaned_end_use_1 = support.parse_end_use(row[version_1])
        ok1 = True
    except:
        cleaned_end_use_1 = 'Failed'
        diff_ok = False
        ok1 = False
    if diff_ok:
        pct_diff = (cleaned_end_use_2 - cleaned_end_use_1) / cleaned_end_use_1
        
        all_diffs[index] = {version_1: cleaned_end_use_1,
                            version_2: cleaned_end_use_2,
                            'diff': pct_diff}
    else:
        failed[index] = {version_1: ok1,
                         version_2: ok2}
        
df_failed = pd.DataFrame(failed).T

In [ ]:
# See the ones that changed
df_failed[df_failed[version_1] != df_failed[version_2]]

In [ ]:
max_diffs = {}
for test, d in all_diffs.items():
    #dmax = 
    max_diffs[test] = {'Max': d['diff'].max().max(),
                       'Min': d['diff'].min().min(),
                       'Total Diff': (d[version_2][('Total', 'kBtu')].sum()
                                      - d[version_1][('Total', 'kBtu')].sum()) / d[version_1][('Total', 'kBtu')].sum()}
    
    
df_diffs = pd.DataFrame(max_diffs).T

In [ ]:
df_diffs[~(df_diffs == 0).all(axis=1)].style.format("{:.2%}")

In [ ]:
from matplotlib.ticker import FuncFormatter

In [ ]:
all_diffs[test]['diff']

In [ ]:
test = ('heatpump_hot_water', 'rb')

fig, ax = plt.subplots(figsize=(16,9))

fmt = lambda x,pos: '{:.0%}'.format(x)

sns.heatmap(all_diffs[test]['diff'].dropna(how='all', axis=0).dropna(how='all', axis=1).abs(),
            ax=ax, cmap='RdYlGn_r',
            #vmin=0, vmax=100,
            cbar_kws={'format': mpl.ticker.FuncFormatter(fmt)},
            annot=all_diffs[test]['diff'].dropna(how='all', axis=0).dropna(how='all', axis=1), fmt='.1%')
ax.set_title("Percent difference in End Use By Fuel for test '{}' between {} and {}".format(test, version_2, version_1))
plt.show()

In [ ]:
cleaned_end_use_2_2_1 = parse_end_use('test/absorption_chillers.rb_2.2.1_out.osw')
cleaned_end_use_2_1_0 = parse_end_use('test/absorption_chillers.rb_2.1.0_out.osw')

In [ ]:
pct_diff = 100*(cleaned_end_use_2_1_0 - cleaned_end_use_2_2_1) / cleaned_end_use_2_1_0

In [ ]:
sns.heatmap(pct_diff.dropna(how='all', axis=0).dropna(how='all', axis=1),
            vmin=0, vmax=100)

In [ ]:
ax = pct_diff.dropna(how='all', axis=0).dropna(how='all', axis=1).plot(kind='bar', subplots=True, figsize=(16,9))
plt.show();

In [ ]:
ax = cleaned_end_use.plot(kind='bar', subplots=True, figsize=(16,9))
plt.show();

In [ ]:
import seaborn as sns
sns.heatmap(cleaned_end_use.dropna(how='all', axis=0).dropna(how='all', axis=1))

In [ ]:
end_use['name'].str.replace('end_use_', '')

# Find missing tests: Map tests to Cpp classes

## Grep in ruby and osm tests

In [ ]:
os.chdir('/home/julien/Software/Others/OpenStudio-resources/model/simulationtests/')

In [ ]:
# Grep in ruby test for Model:: statements
grep = !/bin/grep "Model::" *.rb
objs = pd.DataFrame([x.split(':', maxsplit=1 ) for x in grep], columns=['file', 'grepped_line'])

# Grep in ruby test for Model:: statements
grep_lib = !/bin/grep "Model::" ./lib/*.rb
objs_lib = pd.DataFrame(grep_lib, columns=['grepped_line'])
objs_lib['file'] = 'lib/baseline_model.rb'

# Find all Model namespace Classes by getting name from the cpp files
os_classes = !ls /home/julien/Software/Others/OpenStudio/openstudiocore/src/model/*.cpp
os_classes = [os.path.split(os.path.splitext(p)[0])[1] for p in os_classes]

In [ ]:
model_object_pat = re.compile(r'OpenStudio::Model::(.*?)\.new')
def parse_model_object(s):
    m = model_object_pat.search(s)
    if m:
        return m.groups()[0]
    else:
        print('Cannot match {}'.format(s))
        return None
    
objs['ModelObject'] = objs['grepped_line'].apply(parse_model_object)
objs_lib['ModelObject'] = objs_lib['grepped_line'].apply(parse_model_object)

# Concat both
objs = pd.concat([objs, objs_lib])

In [ ]:
set(objs['ModelObject']) - set(os_classes) 

In [ ]:
# set(os_classes) - set(objs['ModelObject'])

In [ ]:
df_os_classes = pd.DataFrame(index=os_classes)
df_os_classes['In Ruby Test'] = False
df_os_classes = df_os_classes.join(objs.groupby('ModelObject')['file'].apply(list))
df_os_classes.loc[df_os_classes['file'].notnull(),
                  'file'] = df_os_classes.loc[df_os_classes['file'].notnull(),
                                              'file'].apply(np.unique)
df_os_classes.loc[df_os_classes['file'].notnull(), 'In Ruby Test'] = True
df_os_classes = df_os_classes.rename(columns={'file': 'files'})

In [ ]:
df_os_classes['In Ruby Test'].value_counts()

In [ ]:
#df_os_classes.to_csv('Mapping_ruby_test_to_cpp_classes.csv')

## Get comments dict from the google sheet

In [ ]:
comments_dict = {'AccessPolicyStore': 'TRUE',
 'AirLoopHVACReturnPlenum': 'TRUE',
 'AirLoopHVACSupplyPlenum': 'TRUE',
 'AirLoopHVACZoneMixer': 'TRUE',
 'AirLoopHVACZoneSplitter': 'TRUE',
 'AirToAirComponent': 'TRUE',
 'Building': 'TRUE',
 'BuildingStory': 'TRUE',
 'Component': 'TRUE',
 'ComponentData': 'TRUE',
 'ComponentWatcher': 'TRUE',
 'Connection': 'TRUE',
 'ConnectorMixer': 'TRUE',
 'ConstructionBase': 'TRUE',
 'Curve': 'TRUE',
 'CurveBicubic': 'TRUE',
 'CurveDoubleExponentialDecay': 'TRUE',
 'CurveExponentialDecay': 'TRUE',
 'CurveExponentialSkewNormal': 'TRUE',
 'CurveFanPressureRise': 'TRUE',
 'CurveFunctionalPressureDrop': 'TRUE',
 'CurveLinear': 'TRUE',
 'CurveQuadraticLinear': 'TRUE',
 'CurveQuartic': 'TRUE',
 'CurveRectangularHyperbola1': 'TRUE',
 'CurveRectangularHyperbola2': 'TRUE',
 'CurveSigmoid': 'TRUE',
 'CurveTriquadratic': 'TRUE',
 'DefaultConstructionSet': 'TRUE',
 'DefaultScheduleSet': 'TRUE',
 'DefaultSubSurfaceConstructions': 'TRUE',
 'DefaultSurfaceConstructions': 'TRUE',
 'DesignDay': 'TRUE',
 'DesignSpecificationZoneAirDistribution': 'Not in ruby API',
 'ElectricalStorage': 'TRUE',
 'Facility': 'TRUE',
 'FileOperations': 'TRUE',
 'FloorplanJSForwardTranslator': 'TRUE',
 'Gas': 'TRUE',
 'Generator': 'TRUE',
 'GeneratorPhotovoltaic': 'Tested for in photovoltaics.rb',
 'GenericModelObject': 'TRUE',
 'Glazing': 'TRUE',
 'HVACComponent': 'TRUE',
 'Inverter': 'TRUE',
 'LayeredConstruction': 'TRUE',
 'LifeCycleCost': 'It is tested for LifeCycleParameters.rb',
 'LifeCycleCostParameters': 'It is tested for LifeCycleParameters.rb',
 'LifeCycleCostUsePriceEscalation': 'It is tested for LifeCycleParameters.rb',
 'Loop': 'TRUE',
 'Material': 'TRUE',
 'Mixer': 'TRUE',
 'ModelExtensibleGroup': 'TRUE',
 'ModelMerger': 'TRUE',
 'ModelObject': 'TRUE',
 'ModelObjectList': 'TRUE',
 'Node': 'TRUE',
 'OpaqueMaterial': 'True, base class',
 'ParentObject': 'TRUE',
 'PlanarSurface': 'TRUE',
 'PlanarSurfaceGroup': 'TRUE',
 'PortList': 'TRUE',
 'Relationship': 'TRUE',
 'RenderingColor': 'TRUE',
 'ResourceObject': 'TRUE',
 'Schedule': 'TRUE',
 'ScheduleTypeRegistry': 'TRUE',
 'ScheduleWeek': 'TRUE',
 'ScheduleYear': 'TRUE',
 'SetpointManager': 'TRUE',
 'Shade': 'TRUE',
 'Site': 'TRUE',
 'SizingPeriod': 'TRUE',
 'SizingPlant': 'TRUE',
 'SizingSystem': 'TRUE',
 'SizingZone': 'TRUE',
 'Space': 'TRUE',
 'SpaceItem': 'TRUE',
 'SpaceLoad': 'TRUE',
 'SpaceLoadDefinition': 'TRUE',
 'SpaceLoadInstance': 'TRUE',
 'SpaceType': 'TRUE',
 'Splitter': 'TRUE',
 'SubSurface': 'TRUE',
 'Surface': 'TRUE',
 'Thermostat': 'TRUE',
 'ThreeJSForwardTranslator': 'TRUE',
 'ThreeJSReverseTranslator': 'TRUE',
 'UtilityCost_Charge_Block': 'Not Functional in API',
 'UtilityCost_Charge_Simple': 'Not Functional in API',
 'UtilityCost_Computation': 'Not Functional in API',
 'UtilityCost_Qualify': 'Not Functional in API',
 'UtilityCost_Ratchet': 'Not Functional in API',
 'UtilityCost_Tariff': 'Not Functional in API',
 'UtilityCost_Variable': 'Not Functional in API',
 'Version': 'TRUE',
 'WaterToAirComponent': 'TRUE',
 'WaterToWaterComponent': 'TRUE',
 'YearDescription': 'TRUE',
 'ZoneHVACComponent': 'TRUE',
 'ZoneHVACEquipmentList': 'TRUE'}

In [ ]:
from df2gspread import gspread2df as g2d

spreadsheet = '/EffiBEM&NREL-Regression-Test_Status'
wks_name = 'Mapping_ruby_test_to_cpp_classes'

df = g2d.download(spreadsheet, wks_name, col_names = True, row_names = True)
#comments_dict = df['IsNormal'].to_dict()
comments_dict = df.loc[df['IsNormal'] != '', 'IsNormal'].to_dict()
comments_dict

In [ ]:
comments_dict

In [ ]:
#comments.set_index('Test')['IsNormal'].to_dict()

In [ ]:
# Merge comments
comments = pd.Series(comments_dict, name='IsNormal')
df_os_classes = df_os_classes.join(comments)
df_os_classes = df_os_classes[['In Ruby Test', 'IsNormal', 'files']]

## Find objects in the osm tests

In [ ]:
# Compile a regex
os_class_pattern = re.compile(r'OS:(.*?),')

# Initialize a column of empty lists
df_os_classes['osms'] = np.empty((len(df_os_classes), 0)).tolist()

# Loop on all osms, and find OS objects
for osm_path in gb.glob('*.osm'):
    with open(osm_path) as f:
        lines = f.readlines()
    for line in lines:
        m = os_class_pattern.match(line)
        if m:
            classname = m.groups()[0].replace(':','')
            if classname in df.index:
                df_os_classes.loc[classname, 'osms'].append(osm_path)

In [ ]:
df_os_classes.loc[df_os_classes['osms'].apply(len) == 0, 'osms'] = None

In [ ]:
df_os_classes.fillna('')

## Upload to Google

In [ ]:
spreadsheet = '/EffiBEM&NREL-Regression-Test_Status'
wks_name = 'Mapping_ruby_test_to_cpp_classes'
d2g.upload(df_os_classes.fillna(''),
           gfile=spreadsheet, wks_name=wks_name,
           row_names=True, col_names=True)

# Test convergence

In [ ]:
OSCLI = '/home/julien/Software/Others/OS-build/Products/openstudio'
RUN_N_TIMES = 4

## Running the same in.OSW

In [ ]:
os.chdir('/home/julien/Software/Others/OpenStudio-resources/testruns/fan_on_off.rb/')

In [ ]:
!ls

In [ ]:
from subprocess import call
    
r = {}
o = {}
e = {}
for i in range(0, RUN_N_TIMES):
    process = subprocess.Popen([OSCLI, 'run', '-w', 'in.osw'], shell=False,
                           stdout=subprocess.PIPE, 
                           stderr=subprocess.PIPE)

    # wait for the process to terminate
    out, err = process.communicate()
    o[i] = out
    e[i] = err
    errcode = process.returncode
    r[i] = parse_total_site_energy('out.osw')
    print("{} - {:,.0f}".format(i, r[i]))
    
# Say to user
!echo "THIS IS DONE" | espeak

In [ ]:
result = pd.Series(r)

## Running the same test (measure)

In [ ]:
os.chdir('/home/julien/Software/Others/OpenStudio-resources')

In [ ]:
!ls

In [ ]:
diffs = over_5pct.replace(0, np.nan).dropna()['Count (ABS(pct_diff) > 0.005)'].sort_values(ascending=False)

In [ ]:
s = "openstudio model_tests.rb -n '/"
tests = []
for i, (test, ext) in enumerate(diffs[diffs > 1].index.tolist()):
    print(i)
    test_name = "test_{}_{}".format(test, ext)
    #s += " --name test_{}_{}".format(test, ext)
    if i < len(diffs[diffs > 1])-1:
        s+='({})|'.format(test_name)
    else:
        s+='({})'.format(test_name)
    tests.append(test_name)
#print("$os_build/Products/openstudio model_tests.rb {}".format(s))
#print("ruby model_tests.rb {}".format(s))
s += "/'"

In [ ]:
s

In [ ]:
len(diffs[diffs > 1])

In [ ]:
out_dict = {}

In [ ]:
import subprocess

from shutil import copyfile

test_exts = diffs[diffs > 1].index.tolist()
test_exts =  [
    #('surface_properties', 'rb'),
    ('fan_on_off', 'rb')
]
for (test, ext) in test_exts:
    print(test)
 
    #base_path = '/home/julien/Software/Others/OpenStudio-resources/testruns/availability_managers.rb/'
    base_path = '/home/julien/Software/Others/OpenStudio-resources/testruns/{}.{}/'.format(test, ext)


    r = {}
    o = {}
    e = {}
    for i in range(0, 4):
        process = subprocess.Popen([OSCLI, 'model_tests.rb', '-n',
                                    '/{}_{}/'.format(test, ext)], 
                                   shell=False,
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

        # wait for the process to terminate
        out, err = process.communicate()
        o[i] = out
        e[i] = err
        errcode = process.returncode
        
        if errcode != 0:
            print("Problem with {}.{}, run {}".format(test, ext, i))
        else:
            r[i] = parse_total_site_energy(os.path.join(base_path, 'out.osw'))
            print("{} - run {} - {:,.0f}".format(test, i, r[i]))


            # cp the osm somewhere else
            src_path = os.path.join(base_path, 'in.osm')
            dst_path = os.path.join(base_path, '../{t}.{e}_{i}.osm'.format(t=test, e=ext, i=i))
            copyfile(src_path, dst_path)  
            
            src_path = os.path.join(base_path, 'run/in.idf')
            dst_path = os.path.join(base_path, '../{t}.{e}_{i}.idf'.format(t=test, e=ext, i=i))
            copyfile(src_path, dst_path)
    
    out_dict["{}.{}".format(test, ext)] = {'r':r, 'o':o, 'e':e}

!echo "THIS IS DONE" | espeak

In [ ]:
print(out)

In [ ]:
print(err)

In [ ]:
all_dfs = []
for k, v in out_dict.items():
    df = pd.DataFrame(v)
    df['test'] = k
    df['run'] = df.index
    df.set_index(['test', 'run'], inplace=True)
    all_dfs.append(df)

In [ ]:
df_all = pd.concat(all_dfs)

In [ ]:
df_all['r'].unstack(0).pct_change().max().sort_values(ascending=False)*100

In [ ]:
df_all['r'].unstack(0)['surface_properties.rb']

In [ ]:
df_all['r'].unstack(0)['fan_on_off.rb']

In [ ]:
surface_properties.rb, fan_on_off.rb

In [ ]:
 set([".".join([x[0], x[1]]) for x in diffs[diffs > 1].index]) - set(df_all['r'].unstack(0).pct_change().columns)

In [ ]:
result = pd.Series(r)
result

In [ ]:
100*(result - result.iloc[0])/result.iloc[0]

In [ ]:
process = subprocess.Popen([OSCLI, 'model_tests.rb', '-n', '/availability/'], shell=True,
                            stdout=subprocess.PIPE, 
                            stderr=subprocess.PIPE)
out, err = process.communicate()

In [ ]:
out

In [ ]:
r

In [ ]:
r

In [ ]:
o

In [ ]:
e